In [1]:
from library.objective_function import *
from library.optimiser import *
from library.post_analysis import *
from library.experiments import *
%matplotlib inline
SEED = 23191

**naming of objective function**
ackley -> ak 

tuned ackley -> tak

bukin -> bk

egg holder -> eh

**naming of optimizer**

original CMA -> no

line search CMA-> ls

round off CMA-> ro

**naming of experiments**

experiments -> exps + objective function + optimizer + [hyperparameter]

In [3]:
def setup_compsite_cma(inner_opt):
    # One experiment: setup optimizer
    cma = cma_es()
    optmizerParas = {
             'x0': np.zeros((2,)),
             'std': np.ones((2,)) * 3, 
             'tol': 1e-5, 
             'adjust_func': inner_opt, 
             'record': False, 
             'verbose': False}
    cma.set_parameters(optmizerParas)
    return cma
    
def conmpare_efficency_ackley(optimizer, seed):
    np.random.seed(seed)

    # init multiple experiments
    mexps = multiple_experiment()

    # init and setup one experiment
    exp = single_experiment()
    # One experiment: setup objective function
    ak = ackley()
    exp.set_objective(ak)
    exp.set_optimizer(optimizer)

    # Multiple experiment: setup one expreiment
    mexps.set_single_exp(exp)
    # Multiple experiments: setup sample zone of multiple experiments
    zoneParas = {'origin': np.array([0,0]),
             'edge': np.array([500, 500]),
             'step': 100,
             'size': 10,
             'sym': True}
    mexps.set_sample_zone(zoneParas)

    data = mexps.do()
    
    num = zoneParas['edge'][0] / zoneParas['step']
    num = (num + 1) * num / 2
    print("avg probility of converge: ", data['res'].sum() / num)
    print("avg func value: ", data['cost'].sum() / num)
    print("avg evals per exp: ", data['evals'].sum() / num)
    print("\n")

In [29]:
inner_optimizer = do_nothing()
composite_optimizer = setup_compsite_cma(inner_optimizer)
for i in [1291, 3, 100, 29383, 3322, 902, 222]:
    %time conmpare_efficency_ackley(composite_optimizer, i)

avg probility of converge:  0.02666666666666667
avg func value:  19.831645846483443
avg evals per exp:  1480.4799999999998


CPU times: user 14.2 s, sys: 134 ms, total: 14.3 s
Wall time: 14.5 s
avg probility of converge:  0.013333333333333334
avg func value:  20.065190191781713
avg evals per exp:  1470.84


CPU times: user 14 s, sys: 117 ms, total: 14.1 s
Wall time: 14.3 s
diverge, raise LinAlgError!
avg probility of converge:  0.02
avg func value:  19.91107105638078
avg evals per exp:  1493.9200000000003


CPU times: user 14.8 s, sys: 158 ms, total: 15 s
Wall time: 15.1 s
avg probility of converge:  0.03333333333333333
avg func value:  19.675662087266524
avg evals per exp:  1446.6000000000001


CPU times: user 13 s, sys: 70.3 ms, total: 13.1 s
Wall time: 13.2 s
avg probility of converge:  0.02666666666666667
avg func value:  19.74227867506498
avg evals per exp:  1387.2


CPU times: user 12 s, sys: 34.5 ms, total: 12 s
Wall time: 12.1 s
avg probility of converge:  0.02666666666666667
a

# 1. orginal Ackley function

## orginal CMA + original Ackley

In [33]:
inner_optimizer = do_nothing()
composite_optimizer = setup_compsite_cma(inner_optimizer)
%time conmpare_efficency_ackley(composite_optimizer, seed=SEED)

avg probility of converge:  0.02666666666666667
avg func value:  19.821937003313206
avg evals per exp:  1459.48


CPU times: user 12.6 s, sys: 22.9 ms, total: 12.6 s
Wall time: 12.6 s


In [39]:
# time per evalutations (s)
12.6 / 150 / 1459

5.7573680603152835e-05

## line-search-CMA + original Ackley

In [36]:
inner_optimizer = round_off()
composite_optimizer = setup_compsite_cma(inner_optimizer)
%time conmpare_efficency_ackley(composite_optimizer, seed=SEED)

diverge, raise LinAlgError!
avg probility of converge:  0.5066666666666666
avg func value:  9.760844008835326
avg evals per exp:  913.0


CPU times: user 8.36 s, sys: 28.3 ms, total: 8.39 s
Wall time: 8.46 s


In [37]:
8.46 / 913 / 150

6.177437020810515e-05

In [39]:
inner_optimizer = line_search()
composite_optimizer = setup_compsite_cma(inner_optimizer)
%time conmpare_efficency_ackley(composite_optimizer, seed=SEED)

diverge, raise LinAlgError!
avg probility of converge:  0.4133333333333333
avg func value:  11.733333588932046
avg evals per exp:  70947.87333333334


CPU times: user 3min 19s, sys: 1.17 s, total: 3min 20s
Wall time: 3min 28s


In [40]:
228 / 70947 / 150

2.1424443598742724e-05

## adam + original Ackley

In [28]:
ad = adam()
ad.set_parameters(optmizerParas)
%time conmpare_efficency_ackley(ad, seed=SEED)

avg probility of converge:  0.0
avg func value:  19.97863068456147
avg evals per exp:  160.38000000000002


CPU times: user 1.05 s, sys: 10.1 ms, total: 1.06 s
Wall time: 1.06 s


In [29]:
1.06 / 150 / 160

4.416666666666667e-05

## line search + original Ackley

In [30]:
ls = line_search()
%time conmpare_efficency_ackley(ls, seed=SEED)

avg probility of converge:  0.0
avg func value:  19.9743627591425
avg evals per exp:  39.89333333333334


CPU times: user 119 ms, sys: 3.68 ms, total: 123 ms
Wall time: 121 ms


In [31]:
0.121 / 40 / 150

2.0166666666666664e-05

## summary on original ackley
| -- |original CMA | line search - CMA(medium step size) | adam | line search
| -- | -- | -- | -- | -- |
| #experiments| 150 experiments | 150 experiments | 150 experiments | 150 experiments |
| total time| 13.1 s | 180 s | 0.5 s | 0.3 s
| evalutaions per exp | 1.46 k evals |  7.1 k evals |  0.063 k evals | 0.12 k evals
| *performance* | -- | -- | -- | -- |
| probability |  0.03  | 0.41 | 0 | 0 |

# 2. Bukin function

In [33]:
def compare_efficency_bukin(optimizer, seed):
    np.random.seed(seed)

    # init multiple experiments
    mexps = multiple_experiment()

    # init and setup one experiment
    exp = single_experiment()
    # One experiment: setup objective function
    bk = bukin()
    exp.set_objective(bk)
    exp.set_optimizer(optimizer)

    # Multiple experiment: setup one expreiment
    mexps.set_single_exp(exp)
    # Multiple experiments: setup sample zone of multiple experiments
    zoneParas = {'origin': np.array([-15,-15]),
             'edge': np.array([30, 30]),
             'step': 10,
             'size': 10,
             'sym': False}
    mexps.set_sample_zone(zoneParas)

    data = mexps.do()
    
    print("avg probility of converge: ", data['res'].mean())
    print("avg func value: ", data['cost'].mean())
    print("avg evals per exp: ", data['evals'].mean())
    print("\n")
    return data

## adam + bukin func

In [35]:
ad = adam()
%time data = compare_efficency_bukin(ad, seed=SEED)

avg probility of converge:  0.0
avg func value:  11.362796385278592
avg evals per exp:  7224.9


CPU times: user 21 s, sys: 210 ms, total: 21.2 s
Wall time: 21.7 s


In [36]:
21.7 / 90 / 7224

3.3376399655469424e-05

In [ ]:
np.random.seed(23191)

# init multiple experiments
exps_bk_ad = multiple_experiment()

# init and setup one experiment
exp = single_experiment()

# One experiment: setup objective function
bk = bukin()
exp.set_objective(bk)

# One experiment: setup optimizer
ad = adam()
optmizerParas = {'x0': np.zeros((2,)),
         'alpha': 0.01,
         'beta_1': 0.9, 
         'beta_2': 0.999, 
         'epsilon': 1e-8, 
         'max_iter': 1000,
         'tol': 1e-2,              
         'verbose': False,
         'record': False}
ad.set_parameters(optmizerParas)
exp.set_optimizer(ad)

# Multiple experiment: setup one expreiment
exps_bk_ad.set_single_exp(exp)


# Multiple experiments: setup sample zone of multiple experiments
zoneParas = {'origin': np.array([-15, -5]),
             'edge': np.array([30, 10]),
             'step': 10,
             'size': 10,
             'sym': False}
exps_bk_ad.set_sample_zone(zoneParas)

%time data_bk_ad = exps_bk_ad.do()

## original cma + bukin

In [44]:
np.random.seed(23191)

# init multiple experiments
exps3 = multiple_experiment()

# init and setup one experiment
exp = single_experiment()

# One experiment: setup objective function
bk = bukin()
exp.set_objective(bk)

# One experiment: setup optimizer
cma = cma_es()
paras = {'x0': np.zeros((2,)),
         'std': np.ones((2,)) * 0.003, 
         'tol': 1e-5, 
         'adjust_func': do_nothing(), 
         'record': False, 
         'verbose': False}
cma.set_parameters(paras)
exp.set_optimizer(cma)

# Multiple experiment: setup one expreiment
exps3.set_single_exp(exp)


# Multiple experiments: setup sample zone of multiple experiments
expsParas = {'origin': np.array([-15, -5]),
             'edge': np.array([30, 10]),
             'step': 10,
             'size': 10,
             'sym': False}
exps3.set_sample_zone(expsParas)

%time data3 = exps3.do()

avg probility of converge:  0.0
avg func value:  50.832558637424135
avg evals per exp:  998.0


CPU times: user 2.89 s, sys: 34.9 ms, total: 2.92 s
Wall time: 2.97 s


## line-search-cma + bukin

In [ ]:
np.random.seed(2311)

# init multiple experiments
exps3 = multiple_experiment()

# init and setup one experiment
exp = single_experiment()

# One experiment: setup objective function
bk = bukin()
exp.set_objective(bk)

# One experiment: setup optimizer
cma = cma_es()
paras = {'x0': np.zeros((2,)),
         'std': np.ones((2,)) * 0.003, 
         'tol': 1e-5, 
         'adjust_func': line_search(alpha=1e-5, beta=2e-5), 
         'record': False, 
         'verbose': False}
cma.set_parameters(paras)
exp.set_optimizer(cma)

# Multiple experiment: setup one expreiment
exps3.set_single_exp(exp)


# Multiple experiments: setup sample zone of multiple experiments
expsParas = {'origin': np.array([-15, -5]),
             'edge': np.array([30, 10]),
             'step': 10,
             'size': 10,
             'sym': False}
exps3.set_sample_zone(expsParas)

%time data3 = exps3.do()

## summary of bukin
| -- | line search (very small step size) - CMA | original CMA | adam |
| -- | -- | --| -- |
| #experiments| 30 experiments | 30 experiments | 30 experiments |
| total time| 111 s | 4.5 s | 0.7 s |
| evalutaions per exp | 299 k evals |  2.3 k evals | 1 k evals |
| prob | 0.83 | 0 | 0 |

# 3. Tuned Ackley

## original cma + tuned ackley

In [ ]:
np.random.seed(2311)

# init multiple experiments
exps5 = multiple_experiment()

# init and setup one experiment
exp = single_experiment()

# One experiment: setup objective function
tak = tuned_ackley()
exp.set_objective(tak)

# One experiment: setup optimizer
cma = cma_es()
paras = {'mean0': np.zeros((2,)),
         'std': np.ones((2,)) * 3, 
         'tol': 1e-5, 
         'adjust_func': do_nothing(), 
         'record': False, 
         'verbose': False}
cma.set_parameters(paras)
exp.set_optimizer(cma)

# Multiple experiment: setup one expreiment
exps5.set_single_exp(exp)


# Multiple experiments: setup sample zone of multiple experiments
expsParas = {'origin': np.array([0, 0]),
             'edge': np.array([20, 20]),
             'step': 5,
             'size': 10,
             'sym': True}
exps5.set_sample_zone(expsParas)

%time data5 = exps5.do()

## line-searc

In [ ]:
np.random.seed(2311)

# init multiple experiments
exps6 = multiple_experiment()

# init and setup one experiment
exp = single_experiment()

# One experiment: setup objective function
tak = tuned_ackley()
exp.set_objective(tak)

# One experiment: setup optimizer
cma = cma_es()
paras = {'mean0': np.zeros((2,)),
         'std': np.ones((2,)) * 3, 
         'tol': 1e-5, 
         'adjust_func': line_search(), 
         'record': False, 
         'verbose': True}
cma.set_parameters(paras)
exp.set_optimizer(cma)

# Multiple experiment: setup one expreiment
exps6.set_single_exp(exp)


# Multiple experiments: setup sample zone of multiple experiments
expsParas = {'origin': np.array([0, 0]),
             'edge': np.array([20, 20]),
             'step': 5,
             'size': 10,
             'sym': True}
exps6.set_sample_zone(expsParas)

%time data6 = exps6.do()